# Impact of KCs parents on the KC mastery  
In this notebook we will study a toy example of a 3 KCs domain knowledge model. The purpose of this example is to understand the impact parents have on their children.

## Domain knowledge model
We denote A, B and C the three studied KCs and we will suppose they are related with two prerequisite links A -> C and B -> C.

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac
sys.path.append("/home/olivier/PycharmProjects/bayesian-kst/")  # for ubuntu

from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.learner_layer.evaluation import LearnerTrace
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
from sklearn.metrics import roc_auc_score

KC_A = KnowledgeComponent(1, "A")
KC_B = KnowledgeComponent(2, "B")
KC_C = KnowledgeComponent(3, "C")

A_2_C = Link(source=KC_A, target=KC_C)
B_2_C = Link(source=KC_B, target=KC_C)

domain = Domain([KC_A, KC_B, KC_C], [A_2_C, B_2_C])

params = {"slip": .01, "guess":.01}
priors = {'A': 0.1, 'B': 0.1, 'C': 0.1}
# there are 5 exercises corresponding to KC A
ex_1 = Exercise(1, KC_A, "qcm", ex_content="", params=params)
ex_2 = Exercise(2, KC_B, "qcm", ex_content="", params=params)
ex_3 = Exercise(3, KC_C, "qcm", ex_content="", params=params)

## Learner knowledge model
### No prerequisite links
In a first place, in order to differenciate the effect of evaluations and the impact of the prerequisite links, we study how the model behave to learner traces when we don't declare any prerequisite links.

In [2]:
prereqs = {KC_A: {KC_C: 'not existing'}, 
           KC_B: {KC_C: 'not existing'}, 
           KC_C: {KC_A: 'not existing', KC_B:'not existing'}}
learner_pool = LearnerPool(domain, prereqs)

for kc in learner_pool.get_knowledge_components():
    learner_pool.set_learn(kc, .1)
    learner_pool.set_prior(kc, .1)
    learner_pool.set_forget(kc, 0)
    
for ex in [ex_1, ex_2, ex_3]:
    learner_pool.set_slip(ex, .1)
    learner_pool.set_guess(ex, .25)


In [3]:
learner = Learner(1, learner_pool)
answer = [KC_A, True]
learner_traces = [LearnerTrace(learner, ex_1, True)]
print("Results from a succeeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_sequence(learner_traces)
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC A:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  [0.315]


In [4]:
learner = Learner(1, learner_pool)
answer = [KC_A, False]

print("Results from an unsuccueeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC A:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}


AttributeError: 'Learner' object has no attribute 'predict_next_step'

In [ ]:
learner = Learner(1, learner_pool)
answer = [KC_B, True]

print("Results from a succeeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
answer = [KC_B, False]

print("Results from an unsuccueeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
answer = [KC_C, True]

print("Results from a succeeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
answer = [KC_C, False]

print("Results from an unsuccueeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

### Existing prerequisite link structure
Now we study the impact of the prerequisite links, we study how the model behave to learner traces when we don't declare any prerequisite links.

In [ ]:
prereqs = {KC_A: {KC_C: 'strong'}, 
           KC_B: {KC_C: 'weak'}, 
           KC_C: {KC_A: 'strong', KC_B:'weak'}}
learner_pool = LearnerPool(domain, prereqs)

for kc in learner_pool.get_knowledge_components():
    learner_pool.set_learn(kc, .1)
    learner_pool.set_prior(kc, .1)
    learner_pool.set_slip(kc, .1)
    learner_pool.set_guess(kc, .25)
    learner_pool.set_forget(kc, 0)

In [ ]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_A, True]

print("Results from a succeeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_A, False]

print("Results from an unsuccueeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_B, True]

print("Results from a succeeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_B, False]

print("Results from an unsuccueeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_C, True]

print("Results from a succeeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

In [ ]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_C, False]

print("Results from an unsuccueeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)